In [13]:
import tkinter
import os
import tkinter.messagebox
import customtkinter
import pandas as pd
import unicodedata
import requests

# Definindo a aparência padrão como "light"
customtkinter.set_appearance_mode("ligth")
customtkinter.set_default_color_theme("green")

class AppSaudados(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        # Definindo a aparência padrão como "light"
        customtkinter.set_appearance_mode("light")
        customtkinter.set_default_color_theme("green")

        self.title("SAUDADOS")
        self.geometry(f"{1100}x{580}")

        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((1, 2), weight=0)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        self.sidebar_frame = customtkinter.CTkFrame(self, width=140, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)

        #titulo
        self.logo_label = customtkinter.CTkLabel(self.sidebar_frame, text="SAUDADOS – CIÊNCIA DE DADOS E SAÚDE", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        #entrada
        self.Entrada_label = customtkinter.CTkLabel(self.sidebar_frame, text="Olá, Bem Vindo!", font=customtkinter.CTkFont(size=10, weight="bold"))
        self.Entrada_label.grid(row=1, column=0, padx=20, pady=(10))

        #mudar tema
        self.appearance_mode_label = customtkinter.CTkLabel(self.sidebar_frame, text="Aparência:", anchor="w")
        self.appearance_mode_label.grid(row=5, column=0, padx=20, pady=(10, 0))
        self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark", "System"],
                                                                       command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu.grid(row=6, column=0, padx=20, pady=(10, 10))
        self.scaling_label = customtkinter.CTkLabel(self.sidebar_frame, text="Escala:", anchor="w")
        self.scaling_label.grid(row=7, column=0, padx=20, pady=(10, 0))
        self.scaling_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["80%", "90%", "100%", "110%", "120%"],
                                                               command=self.change_scaling_event)
        self.scaling_optionemenu.grid(row=8, column=0, padx=20, pady=(10, 20))

        # create caixa de texto
        self.caixaTexto = customtkinter.CTkTextbox(self, width=100)
        self.caixaTexto.grid(row=0, column=1, padx=(20, 0), pady=(20, 0), sticky="nsew")
        self.caixaTexto.insert("0.0", "SAUDADOS\n\n" +"O objetivo desse sistema é apoiar as ações de pesquisa utilizando dados de bases da saúde, bem como dados demográficos entre outros.\n\n" + "Instruções:\n\n")

        #base de dados
        self.label_base_dados = customtkinter.CTkLabel(self, text="Selecione a Base de Dados:")
        self.label_base_dados.grid(row=1, column=1, padx=20, pady=(10, 0))

        # Obter todos os arquivos de um diretório específico
        diretorio = 'D:\Area de Trabalho\Repositorys\PIBITI-IFBA-CNPq-2023-2024\ProjetoSaudados\BaseDadosTeste-Saudados'  
        arquivos = [arq for arq in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, arq))]

        self.menu_base_dados = customtkinter.CTkOptionMenu(self, values=arquivos, command=self)
        self.menu_base_dados.grid(row=2, column=1, padx=20, pady=(10, 10))

        #EXTENSAO
        self.label_extensao = customtkinter.CTkLabel(self, text="Selecione a Extensão:")
        self.label_extensao.grid(row=3, column=1, padx=20, pady=(10, 0))
        self.menu_extensao = customtkinter.CTkOptionMenu(self, values=[".csv", ".xlsx", ".json", ".dbf", ".txt"], command=self)
        self.menu_extensao.grid(row=4, column=1, padx=20, pady=(10, 10))

        #Botao ler arquivo
        self.button_LerArquivo = customtkinter.CTkButton(self, text="Ler Arquivo", command=self.on_button_click)
        self.button_LerArquivo.grid(row=5, column=1, padx=20, pady=10)

        #Debug
        self.entry = customtkinter.CTkEntry(self, placeholder_text="BEM VINDOS!")
        self.entry.grid(row=7, column=1, columnspan=2, padx=(20, 0), pady=(20, 20), sticky="nsew")

        # create tabview
        self.scrollable_frame = customtkinter.CTkScrollableFrame(self, label_text="Selecione as Colunas:")
        self.scrollable_frame.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        self.scrollable_frame.grid_columnconfigure(0, weight=1)
        self.scrollable_frame_switches = []
        # for i in range(100):
        #     switch = customtkinter.CTkSwitch(master=self.scrollable_frame, text=f"Coluna {i}")
        #     switch.grid(row=i, column=0, padx=10, pady=(0, 20))
        #     self.scrollable_frame_switches.append(switch)

    def change_appearance_mode_event(self, new_appearance_mode):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)
    
    def remove_special_characters(text):
        text = ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))
        return text.upper()

    def on_button_click(self):
        selected_extension = self.menu_extensao.get()
        arquivo = self.menu_base_dados.get()
        self.entry.delete(0, tkinter.END)
        self.entry.insert(0, f"Lendo arquivo {arquivo}...")

        try:
            if selected_extension == ".csv":
                df = pd.read_csv(arquivo, delimiter=self.determinar_delimitador(arquivo), low_memory=False)
            elif selected_extension == ".xlsx":
                df = pd.read_excel(arquivo, delimiter=self.determinar_delimitador(arquivo), low_memory=False)
            elif selected_extension == ".json":
                df = pd.read_json(arquivo, delimiter=self.determinar_delimitador(arquivo), low_memory=False)

            self.entry.delete(0, tkinter.END)
            self.entry.insert(0, "Arquivo carregado com sucesso!")

            # Limpar os interruptores existentes
            for widget in self.scrollable_frame.winfo_children():
                widget.destroy()

            # Criar interruptores para cada coluna do DataFrame
            columns = df.columns
            for i, col in enumerate(columns):
                switch = customtkinter.CTkSwitch(master=self.scrollable_frame, text=f"{col}")
                switch.grid(row=i, column=0, padx=10, pady=(0, 20))
                self.scrollable_frame_switches.append(switch)

        except Exception as e:
            self.entry.delete(0, tkinter.END)
            self.entry.insert(0, f"Erro ao carregar arquivo: {e}")

    def determinar_delimitador(self, arquivo):
        # Aqui, você deve decidir como encontrar o delimitador com base no arquivo selecionado
        # Se for um arquivo .csv, por exemplo, você pode abrir o arquivo e analisar a primeira linha para encontrar o delimitador
        # O exemplo a seguir considera que o arquivo selecionado é um .csv e usa uma abordagem simples para encontrar o delimitador
        with open(arquivo, 'r') as file:
            line = file.readline()
            delimiters = [',', ';', '\t']
            for delimiter in delimiters:
                if delimiter in line:
                    return delimiter
            return ','

if __name__ == "__main__":
    app = AppSaudados()
    app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "d:\Users\Eduarda\anaconda3\Lib\site-packages\customtkinter\windows\widgets\core_widget_classes\dropdown_menu.py", line 101, in <lambda>
    command=lambda v=value: self._button_callback(v),
                            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Users\Eduarda\anaconda3\Lib\site-packages\customtkinter\windows\widgets\core_widget_classes\dropdown_menu.py", line 106, in _button_callback
    self._command(value)
  File "d:\Users\Eduarda\anaconda3\Lib\site-packages\customtkinter\windows\widgets\ctk_optionmenu.py", line 385, in _dropdown_callback
    self._command(self._current_value)
TypeError: 'AppSaudados' object is not callable


KeyboardInterrupt: 